In [1]:
from IPython.core.display import display,HTML

display(HTML("<style>.container { width: 90% !important;}</style>"))

/var/folders/s2/bjtyvybn0kv3rwjkn5xht9_m0000gn/T/ipykernel_5469/2814205877.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
import pandas as pd
import govt_func as gf
import datetime
from imp import reload

import numpy as np
from dateutil.relativedelta import *

# Import Data

In [3]:
deferred_uri = "data/deferred_income.xlsx"
df_deferred = pd.read_excel(deferred_uri)
profit_uri = "data/profit.xlsx"
df_profit = pd.read_excel(profit_uri)
equity_uri = "data/total_equity.xlsx"
df_equity = pd.read_excel(equity_uri)

df_deferred = df_deferred.drop("证券名称", axis=1)
df_profit = df_profit.drop("证券名称", axis=1)
df_equity = df_equity.drop("证券名称", axis=1)

df_deferred = df_deferred.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_profit = df_profit.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_equity = df_equity.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})

df_deferred = df_deferred.dropna(axis=0, subset="INDUSTRY")
df_profit = df_profit.dropna(axis=0, subset="INDUSTRY")
df_equity = df_equity.dropna(axis=0, subset="INDUSTRY")

df_deferred = df_deferred.drop(
    df_deferred.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_profit = df_profit.drop(
    df_profit.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement

df_equity = df_equity.drop(
    df_equity.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_equity

,SECU_CODE,INDUSTRY,2015Q1,2015Q2,2015Q3,2016Q1,2016Q2,2016Q3,2017Q1,2017Q2,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
0,000001.SZ,银行,1.367040e+11,1.508800e+11,1.571360e+11,1.875690e+11,1.917000e+11,1.982280e+11,2.077390e+11,2.114540e+11,...,2.880730e+11,3.523550e+11,3.513970e+11,3.587710e+11,3.726170e+11,3.771930e+11,3.888580e+11,4.061750e+11,4.120980e+11,4.253840e+11
1,000002.SZ,房地产,1.162035e+11,1.177999e+11,1.211252e+11,1.387489e+11,1.381740e+11,1.435336e+11,1.619860e+11,1.611578e+11,...,2.448519e+11,2.741111e+11,2.912561e+11,3.026424e+11,3.554320e+11,3.637208e+11,3.749304e+11,3.933097e+11,3.917086e+11,4.059685e+11
2,000004.SZ,计算机,1.556272e+08,1.617951e+08,1.638883e+08,1.604421e+08,1.662884e+08,1.705901e+08,1.765099e+08,1.724439e+08,...,1.175867e+08,1.410845e+09,1.408951e+09,1.461701e+09,1.461128e+09,1.462960e+09,1.490778e+09,9.352980e+08,9.122587e+08,8.904506e+08
3,000005.SZ,电力及公用事业,6.616485e+08,6.465265e+08,6.398790e+08,1.206778e+09,1.344563e+09,1.355084e+09,1.345628e+09,1.340277e+09,...,1.629619e+09,1.630011e+09,1.708784e+09,1.714528e+09,1.341020e+09,1.524917e+09,1.525147e+09,1.475787e+09,1.482050e+09,1.481608e+09
4,000006.SZ,房地产,4.425511e+09,4.324681e+09,4.342856e+09,4.544098e+09,4.437854e+09,4.452352e+09,5.242471e+09,5.150181e+09,...,6.708967e+09,7.310838e+09,7.166013e+09,7.461954e+09,8.122229e+09,7.983140e+09,8.170081e+09,8.229441e+09,8.261423e+09,8.236666e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5005,688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.749009e+08,6.088049e+08,6.965272e+08,7.322165e+08,1.421966e+09,1.584984e+09,1.566872e+09,1.603426e+09
5006,688800.SH,电子,NaN,1.814390e+08,NaN,NaN,2.206744e+08,NaN,NaN,3.741731e+08,...,NaN,NaN,5.091657e+08,NaN,5.504010e+08,5.707413e+08,9.553454e+08,1.053107e+09,1.088638e+09,1.830459e+09
5007,688819.SH,电力设备及新能源,NaN,2.745337e+09,NaN,NaN,3.409943e+09,NaN,NaN,4.362357e+09,...,NaN,NaN,5.655621e+09,6.377500e+09,1.227036e+10,1.206134e+10,1.246827e+10,1.316418e+10,1.289788e+10,1.356150e+10
5008,688981.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.643923e+09,7.238543e+10,9.330745e+10,1.434151e+11,1.456262e+11,1.470942e+11,1.514041e+11,1.672907e+11,1.832109e+11,1.985105e+11


# Get DEFERRED in Quarter 

In [4]:
df = df_deferred.set_index(["SECU_CODE", "INDUSTRY"])

df = df.stack()
df = df.reset_index()

df_deferred = df.rename(columns={"level_2": "QUARTER", 0: "DEFERRED"})
df_deferred

,SECU_CODE,INDUSTRY,QUARTER,DEFERRED
0,000004.SZ,计算机,2020Q1,624527.72
1,000004.SZ,计算机,2020Q2,624527.72
2,000004.SZ,计算机,2020Q3,624527.72
3,000004.SZ,计算机,2021Q1,471532.49
4,000004.SZ,计算机,2021Q2,471532.49
...,...,...,...,...
75447,689009.SH,机械,2021Q2,45201982.08
75448,689009.SH,机械,2021Q3,44515389.58
75449,689009.SH,机械,2022Q1,46142204.58
75450,689009.SH,机械,2022Q2,45270629.01


In [5]:
df_deferred = df_deferred.sort_values(by=["QUARTER", "SECU_CODE"])
df_deferred

,SECU_CODE,INDUSTRY,QUARTER,DEFERRED
35,000009.SZ,综合,2015Q1,2.036336e+08
59,000011.SZ,房地产,2015Q1,2.112636e+07
70,000012.SZ,建材,2015Q1,4.325027e+08
94,000016.SZ,家电,2015Q1,1.520777e+08
118,000019.SZ,农林牧渔,2015Q1,1.436657e+07
...,...,...,...,...
75410,688799.SH,医药,2022Q3,5.327174e+07
75424,688800.SH,电子,2022Q3,3.536078e+07
75434,688819.SH,电力设备及新能源,2022Q3,5.976020e+08
75443,688981.SH,电子,2022Q3,4.116338e+09


# Get deferred_q
# ${deferred_q}_i = deferred_i /4 $

In [6]:
df_deferred_q = df_deferred.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"]) / 4
df_deferred_q = df_deferred_q.rename(columns={"DEFERRED": "DEFERRED_Q"})
df_deferred_q

DEFERRED_Q
QUARTER SECU_CODE INDUSTRY              
2015Q1  000009.SZ 综合        5.090839e+07
        000011.SZ 房地产       5.281591e+06
        000012.SZ 建材        1.081257e+08
        000016.SZ 家电        3.801942e+07
        000019.SZ 农林牧渔      3.591643e+06
...                                  ...
2022Q3  688799.SH 医药        1.331794e+07
        688800.SH 电子        8.840195e+06
        688819.SH 电力设备及新能源  1.494005e+08
        688981.SH 电子        1.029084e+09
        689009.SH 机械        1.115097e+07

[75452 rows x 1 columns]

# Plug in equity data

In [7]:
df = df_equity.set_index(["SECU_CODE", "INDUSTRY"])
df_equity = df.stack().reset_index().rename(columns={"level_2": "QUARTER", 0: "EQUITY"})
df_equity

,SECU_CODE,INDUSTRY,QUARTER,EQUITY
0,000001.SZ,银行,2015Q1,1.367040e+11
1,000001.SZ,银行,2015Q2,1.508800e+11
2,000001.SZ,银行,2015Q3,1.571360e+11
3,000001.SZ,银行,2016Q1,1.875690e+11
4,000001.SZ,银行,2016Q2,1.917000e+11
...,...,...,...,...
93177,689009.SH,机械,2021Q2,4.021538e+09
93178,689009.SH,机械,2021Q3,4.181275e+09
93179,689009.SH,机械,2022Q1,4.336026e+09
93180,689009.SH,机械,2022Q2,4.596629e+09


# plug in profit

In [8]:
df = df_profit.set_index(["SECU_CODE", "INDUSTRY"])
df_profit = gf.get_quarter(df)
df_profit

,,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,2013Q3,2014Q1,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
SECU_CODE,INDUSTRY,,,,,,,,,,,,,,,,,,,,,
000001.SZ,银行,3.069099e+09,2.996747e+09,3.858597e+09,4.477351e+09,4.401298e+09,4.533465e+09,4.718000e+09,5.188000e+09,5.479000e+09,6.697000e+09,...,1.067300e+10,1.095800e+10,6.629000e+09,1.123900e+10,1.287400e+10,9.467000e+09,1.444600e+10,1.618400e+10,1.159900e+10,1.832900e+10
000002.SZ,房地产,1.674455e+09,2.777354e+09,1.198387e+09,2.058143e+09,3.990529e+09,2.117311e+09,2.394886e+09,4.738527e+09,2.867622e+09,2.039198e+09,...,1.409881e+10,4.795679e+09,2.165477e+10,1.538755e+10,4.108071e+09,1.797280e+10,1.102043e+10,4.081142e+09,2.027806e+10,1.337192e+10
000004.SZ,计算机,2.080828e+06,2.616066e+06,2.132602e+06,1.558349e+06,9.595499e+05,3.018656e+06,3.720557e+06,2.269117e+06,4.276827e+06,-2.974611e+06,...,-8.795067e+06,4.403971e+06,-9.140229e+05,5.702308e+07,4.846545e+06,1.922285e+06,2.767515e+07,-1.309889e+07,-2.791890e+07,-2.532426e+07
000005.SZ,电力及公用事业,-6.291033e+06,3.341440e+07,-1.346671e+07,-1.222558e+07,-1.703261e+07,1.587058e+07,-9.235390e+06,-1.322147e+07,-6.543229e+06,-8.682228e+06,...,-1.663681e+07,-2.230184e+07,-1.106386e+07,5.769103e+06,8.595700e+06,1.919936e+08,2.697103e+06,8.476993e+05,2.653019e+06,3.613460e+06
000006.SZ,房地产,7.129346e+07,1.175649e+08,1.714304e+08,1.187848e+08,2.667864e+08,9.885507e+07,8.335351e+07,2.713965e+08,1.207738e+08,4.036137e+07,...,3.318325e+08,1.829440e+08,1.227008e+08,3.177867e+08,3.655433e+08,1.401563e+08,2.297127e+08,5.677890e+07,2.596736e+08,-3.585225e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.915974e+07,3.472115e+07,3.863758e+07,3.881189e+07,4.102057e+07,4.055120e+07,3.984571e+07,5.573177e+07,4.200342e+07
688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.999367e+06,3.943728e+07,2.068451e+07,1.874455e+07,2.348991e+07,3.763935e+07,6.377653e+07,7.939895e+07,8.094975e+07
688819.SH,电力设备及新能源,NaN,1.465038e+08,2.644794e+08,2.226396e+08,2.627831e+08,2.908038e+08,1.371183e+08,-1.443808e+07,1.287562e+08,6.622459e+07,...,5.615807e+08,4.031860e+08,5.723244e+08,8.695559e+08,5.226400e+08,3.186100e+08,4.663662e+08,4.854114e+08,3.768847e+08,8.147876e+08


In [9]:
df = df_profit.stack()
df = df.reset_index()
df = df.rename(columns={"level_2": "QUARTER", 0: "PROFIT"})
df_profit = df.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])  # synchronize index
df_profit

,,,PROFIT
QUARTER,SECU_CODE,INDUSTRY,
2011Q1,000001.SZ,银行,3.069099e+09
2011Q2,000001.SZ,银行,2.996747e+09
2011Q3,000001.SZ,银行,3.858597e+09
2012Q1,000001.SZ,银行,4.477351e+09
2012Q2,000001.SZ,银行,4.401298e+09
...,...,...,...
2021Q2,689009.SH,机械,3.214818e+08
2021Q3,689009.SH,机械,1.860397e+08
2022Q1,689009.SH,机械,7.554774e+07


# $ support_i = deferred_i/equity_i$

In [10]:
df_deferred = df_deferred.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df_equity = df_equity.set_index(["QUARTER", "SECU_CODE", "INDUSTRY"])
df = pd.merge(
    left=df_deferred, right=df_equity, left_index=True, right_index=True
)  # merge the dataframe, make deferred and equity in the same table
df

DEFERRED        EQUITY
QUARTER SECU_CODE INDUSTRY                            
2015Q1  000009.SZ 综合        2.036336e+08  5.585840e+09
        000011.SZ 房地产       2.112636e+07  2.076333e+09
        000012.SZ 建材        4.325027e+08  8.799920e+09
        000016.SZ 家电        1.520777e+08  4.366765e+09
        000019.SZ 农林牧渔      1.436657e+07  9.867472e+08
...                                  ...           ...
2022Q3  688799.SH 医药        5.327174e+07  1.603426e+09
        688800.SH 电子        3.536078e+07  1.830459e+09
        688819.SH 电力设备及新能源  5.976020e+08  1.356150e+10
        688981.SH 电子        4.116338e+09  1.985105e+11
        689009.SH 机械        4.460389e+07  4.783259e+09

[75452 rows x 2 columns]

In [11]:
df["F_SUPPORT"] = df["DEFERRED"] / df["EQUITY"]
df

DEFERRED        EQUITY  F_SUPPORT
QUARTER SECU_CODE INDUSTRY                                       
2015Q1  000009.SZ 综合        2.036336e+08  5.585840e+09   0.036455
        000011.SZ 房地产       2.112636e+07  2.076333e+09   0.010175
        000012.SZ 建材        4.325027e+08  8.799920e+09   0.049148
        000016.SZ 家电        1.520777e+08  4.366765e+09   0.034826
        000019.SZ 农林牧渔      1.436657e+07  9.867472e+08   0.014560
...                                  ...           ...        ...
2022Q3  688799.SH 医药        5.327174e+07  1.603426e+09   0.033224
        688800.SH 电子        3.536078e+07  1.830459e+09   0.019318
        688819.SH 电力设备及新能源  5.976020e+08  1.356150e+10   0.044066
        688981.SH 电子        4.116338e+09  1.985105e+11   0.020736
        689009.SH 机械        4.460389e+07  4.783259e+09   0.009325

[75452 rows x 3 columns]

# $dependence_i = {deferred_q}_i /profit_i$

In [12]:
df1 = pd.merge(
    left=df_deferred_q, right=df_profit, left_index=True, right_index=True
)  # merge the dataframe and make DEFERRED_Q and PROFIT in the same table
df1

DEFERRED_Q        PROFIT
QUARTER SECU_CODE INDUSTRY                            
2015Q1  000009.SZ 综合        5.090839e+07  1.270414e+08
        000011.SZ 房地产       5.281591e+06  2.019918e+06
        000012.SZ 建材        1.081257e+08  9.107638e+07
        000016.SZ 家电        3.801942e+07  3.176376e+07
        000019.SZ 农林牧渔      3.591643e+06 -1.069070e+07
...                                  ...           ...
2022Q3  688799.SH 医药        1.331794e+07  4.200342e+07
        688800.SH 电子        8.840195e+06  8.094975e+07
        688819.SH 电力设备及新能源  1.494005e+08  8.147876e+08
        688981.SH 电子        1.029084e+09  3.909379e+09
        689009.SH 机械        1.115097e+07  1.828479e+08

[75388 rows x 2 columns]

In [13]:
df1["DEPENDENCE"] = df1["DEFERRED_Q"] / df1["PROFIT"]
# subsidy > profit : dependence = 1
# other: original value
df1["F_DEPENDENCE"] = df1.DEPENDENCE
df1["F_DEPENDENCE"] = df1.F_DEPENDENCE.mask((df1.DEFERRED_Q > df1.PROFIT), 1)
df1

DEFERRED_Q        PROFIT  DEPENDENCE  \
QUARTER SECU_CODE INDUSTRY                                           
2015Q1  000009.SZ 综合        5.090839e+07  1.270414e+08    0.400723   
        000011.SZ 房地产       5.281591e+06  2.019918e+06    2.614755   
        000012.SZ 建材        1.081257e+08  9.107638e+07    1.187198   
        000016.SZ 家电        3.801942e+07  3.176376e+07    1.196943   
        000019.SZ 农林牧渔      3.591643e+06 -1.069070e+07   -0.335959   
...                                  ...           ...         ...   
2022Q3  688799.SH 医药        1.331794e+07  4.200342e+07    0.317068   
        688800.SH 电子        8.840195e+06  8.094975e+07    0.109206   
        688819.SH 电力设备及新能源  1.494005e+08  8.147876e+08    0.183361   
        688981.SH 电子        1.029084e+09  3.909379e+09    0.263235   
        689009.SH 机械        1.115097e+07  1.828479e+08    0.060985   

                            F_DEPENDENCE  
QUARTER SECU_CODE INDUSTRY                
2015Q1  000009.SZ 综合            0.400723  
        000011.SZ 房地产           1.000000  
        000012.SZ 建材            1.000000  
        000016.SZ 家电            1.000000  
        000019.SZ 农林牧渔          1.000000  
...                                  ...  
2022Q3  688799.SH 医药            0.317068  
        688800.SH 电子            0.109206  
        688819.SH 电力设备及新能源      0.183361  
        688981.SH 电子            0.263235  
        689009.SH 机械            0.060985  

[75388 rows x 4 columns]

In [14]:
df = pd.merge(
    left=df, right=df1, left_index=True, right_index=True
)  # merge the dataframe
df_deferred = df.drop(
    ["DEFERRED", "EQUITY", "PROFIT", "DEPENDENCE", "DEFERRED_Q"], axis=1
)
df_deferred

F_SUPPORT  F_DEPENDENCE
QUARTER SECU_CODE INDUSTRY                         
2015Q1  000009.SZ 综合         0.036455      0.400723
        000011.SZ 房地产        0.010175      1.000000
        000012.SZ 建材         0.049148      1.000000
        000016.SZ 家电         0.034826      1.000000
        000019.SZ 农林牧渔       0.014560      1.000000
...                               ...           ...
2022Q3  688799.SH 医药         0.033224      0.317068
        688800.SH 电子         0.019318      0.109206
        688819.SH 电力设备及新能源   0.044066      0.183361
        688981.SH 电子         0.020736      0.263235
        689009.SH 机械         0.009325      0.060985

[75388 rows x 2 columns]

# Rank F_SUPPORT & F_DEPENDENCE

In [15]:
df_deferred = df_deferred.groupby(by="QUARTER").rank(axis=0)
df_deferred

F_SUPPORT  F_DEPENDENCE
QUARTER SECU_CODE INDUSTRY                         
2015Q1  000009.SZ 综合           1567.0        1123.0
        000011.SZ 房地产           834.0        1674.5
        000012.SZ 建材           1700.0        1674.5
        000016.SZ 家电           1539.0        1674.5
        000019.SZ 农林牧渔         1021.0        1674.5
...                               ...           ...
2022Q3  688799.SH 医药           3474.0        2445.0
        688800.SH 电子           2881.0        1687.0
        688819.SH 电力设备及新能源     3669.0        2090.0
        688981.SH 电子           2963.0        2343.0
        689009.SH 机械           2019.0        1272.0

[75388 rows x 2 columns]

# Standardization

In [16]:
df = df_deferred
mean = df.groupby("QUARTER").mean()
std = df.groupby("QUARTER").std()
df = gf.get_standard(df, mean, std)
df

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_SUPPORT_mean,F_DEPENDENCE_mean,F_SUPPORT_std,F_DEPENDENCE_std
QUARTER,,,,,,,,
2015Q1,000009.SZ,综合,1.016686,0.241134,987.5,987.5,569.989035,561.929144
2015Q1,000011.SZ,房地产,-0.269303,1.222574,987.5,987.5,569.989035,561.929144
2015Q1,000012.SZ,建材,1.250024,1.222574,987.5,987.5,569.989035,561.929144
2015Q1,000016.SZ,家电,0.967562,1.222574,987.5,987.5,569.989035,561.929144
2015Q1,000019.SZ,农林牧渔,0.058773,1.222574,987.5,987.5,569.989035,561.929144
...,...,...,...,...,...,...,...,...
2022Q3,688799.SH,医药,1.198285,0.333809,2053.5,2053.5,1185.444431,1172.824794
2022Q3,688800.SH,电子,0.698050,-0.312493,2053.5,2053.5,1185.444431,1172.824794
2022Q3,688819.SH,电力设备及新能源,1.362780,0.031121,2053.5,2053.5,1185.444431,1172.824794


In [17]:
df_deferred = df.drop(["F_SUPPORT_mean", "F_DEPENDENCE_mean", "F_SUPPORT_std", "F_DEPENDENCE_std"],axis = 1)
df_deferred

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE
QUARTER,,,,
2015Q1,000009.SZ,综合,1.016686,0.241134
2015Q1,000011.SZ,房地产,-0.269303,1.222574
2015Q1,000012.SZ,建材,1.250024,1.222574
2015Q1,000016.SZ,家电,0.967562,1.222574
2015Q1,000019.SZ,农林牧渔,0.058773,1.222574
...,...,...,...,...
2022Q3,688799.SH,医药,1.198285,0.333809
2022Q3,688800.SH,电子,0.698050,-0.312493
2022Q3,688819.SH,电力设备及新能源,1.362780,0.031121


# Exclude Outlier

In [18]:
df_deferred[df_deferred.F_SUPPORT > 3] = np.nan
df_deferred[df_deferred.F_SUPPORT < -3] = np.nan
df_deferred[df_deferred.F_DEPENDENCE > 3] = np.nan
df_deferred[df_deferred.F_DEPENDENCE < -3] = np.nan
df_deferred

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE
QUARTER,,,,
2015Q1,000009.SZ,综合,1.016686,0.241134
2015Q1,000011.SZ,房地产,-0.269303,1.222574
2015Q1,000012.SZ,建材,1.250024,1.222574
2015Q1,000016.SZ,家电,0.967562,1.222574
2015Q1,000019.SZ,农林牧渔,0.058773,1.222574
...,...,...,...,...
2022Q3,688799.SH,医药,1.198285,0.333809
2022Q3,688800.SH,电子,0.698050,-0.312493
2022Q3,688819.SH,电力设备及新能源,1.362780,0.031121


# Get F_DEFERRED

In [19]:
df_deferred["F_DEFERRED"] = df_deferred["F_SUPPORT"] - df_deferred["F_DEPENDENCE"]
df_deferred

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_DEFERRED
QUARTER,,,,,
2015Q1,000009.SZ,综合,1.016686,0.241134,0.775553
2015Q1,000011.SZ,房地产,-0.269303,1.222574,-1.491877
2015Q1,000012.SZ,建材,1.250024,1.222574,0.027450
2015Q1,000016.SZ,家电,0.967562,1.222574,-0.255012
2015Q1,000019.SZ,农林牧渔,0.058773,1.222574,-1.163801
...,...,...,...,...,...
2022Q3,688799.SH,医药,1.198285,0.333809,0.864475
2022Q3,688800.SH,电子,0.698050,-0.312493,1.010544
2022Q3,688819.SH,电力设备及新能源,1.362780,0.031121,1.331659


In [20]:
df = df_deferred.reset_index()
df = df.sort_values(by = ["QUARTER","SECU_CODE"])
df = df.set_index("QUARTER")
df

,SECU_CODE,INDUSTRY,F_SUPPORT,F_DEPENDENCE,F_DEFERRED
QUARTER,,,,,
2015Q1,000009.SZ,综合,1.016686,0.241134,0.775553
2015Q1,000011.SZ,房地产,-0.269303,1.222574,-1.491877
2015Q1,000012.SZ,建材,1.250024,1.222574,0.027450
2015Q1,000016.SZ,家电,0.967562,1.222574,-0.255012
2015Q1,000019.SZ,农林牧渔,0.058773,1.222574,-1.163801
...,...,...,...,...,...
2022Q3,688799.SH,医药,1.198285,0.333809,0.864475
2022Q3,688800.SH,电子,0.698050,-0.312493,1.010544
2022Q3,688819.SH,电力设备及新能源,1.362780,0.031121,1.331659


# Write File

In [21]:
df.to_csv("cleaned_data/cleaned_deferred.csv")